# Tarea para el Hogar 02

Esta Tarea para el Hogar 02 se entrega el final de la segunda clase
<br> se espera de usted que intente avanzar con los desafios propuestos y que los traiga terminados para la Clase 03, ya que se analizarán los resultados

##  1. Ensembles de Modelos

Vea el siguiente video [BBC - The Code - The Wisdom of the Crowd](https://www.youtube.com/watch?v=iOucwX7Z1HU)    ( 5 min)


Lea los siguientes artículos


*   [The Wisdom of Crowds (Vox Populi) by Francis Galton](https://www.all-about-psychology.com/the-wisdom-of-crowds.html)  (10 min)
*   [A Gentle Introduction to Ensemble Learning](https://machinelearningmastery.com/what-is-ensemble-learning/)  (10 min)





---



##  2.  Zero2Hero   primera parte
Se han lanzado los primeros fascículos coleccionables llamados "from Zero to Hero" que muy detalladamente, paso a paso enseñan todo lo necesario de R para entender los scripts oficiales de la asignatura.
Están en el repositorio oficial de la asignatura, carpeta  **src/zero2hero**



---



## 3.  Grid Search

Busque en internet el precido significado de los hiperparámetros de la librería **rpart**  que está implementando el algoritmo **CART**  Classification and Regression Trees  propuesto en el año 1984 por Leo Brieman:

*   cp
*   maxdepth
*   minsplit
*   minbucket

Entienda que valores es razonable tome cada hiperparámetro,  en particular profundice en el hiperparámetro  **cp**  y la posibilidad que tome valores negativos.  Es válido consultar a su amigo de *capacidades especiales*  ChatGPT


En las siguientes celdas a un notebook incompleto, un esqueleto de codigo brindado a modo de facilitarle la tarea de codeo y permitir que su valiosa cognición se concentre temas conceptuales de Ciencia de Datos

Modifiquelo agregando loops para que recorra TODOS los hiperparámetros de rpart  < cp, maxdepth, minsplit, minbucket >, y luego póngalo a correr. Recuerde cambiar por SU semilla
Tenga muy presente la granularidad que eligirá para cada hiperparámetro.

### Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Tipe -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [1]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Drive already mounted at /content/.drive; to attempt to forcibly remount, call drive.mount("/content/.drive", force_remount=True).


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [2]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/austral2025-af91/dataset_pequeno.csv"
archivo_destino="/content/datasets/dataset_pequeno.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/dataset_pequeno.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi



ln: failed to create symbolic link '/content/buckets/b1/labo1': File exists


limpio el ambiente de R

In [1]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,657177,35.1,1467094,78.4,1467094,78.4
Vcells,1220152,9.4,8388608,64.0,1975128,15.1


In [2]:
# cargo las librerias que necesito
require("data.table")
require("rpart")
require("parallel")
if (!require("primes")) install.packages("primes")
require("primes")

Loading required package: data.table

Loading required package: rpart

Loading required package: parallel

Loading required package: primes



Aqui debe poner SU semiila primigenia

In [3]:
PARAM <- list()
# reemplazar por su primer semilla
PARAM$semilla_primigenia <- 765179
PARAM$qsemillas <- 10

PARAM$training_pct <- 70L  # entre  1L y 99L

# elegir SU dataset comentando/ descomentando
PARAM$dataset_nom <- "~/datasets/dataset_pequeno.csv"

In [4]:
# particionar agrega una columna llamada fold a un dataset
#  que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30), agrupa=clase_ternaria, seed=semilla)
#   crea una particion 70, 30

particionar <- function(data, division, agrupa = "", campo = "fold", start = 1, seed = NA) {
  if (!is.na(seed)) set.seed(seed)

  bloque <- unlist(mapply(function(x, y) {
    rep(y, x)
  }, division, seq(from = start, length.out = length(division))))

  data[, (campo) := sample(rep(bloque, ceiling(.N / length(bloque))))[1:.N],
    by = agrupa
  ]
}

In [5]:
ArbolEstimarGanancia <- function(semilla, training_pct, param_basicos) {
  # particiono estratificadamente el dataset
  particionar(dataset,
    division = c(training_pct, 100L -training_pct),
    agrupa = "clase_ternaria",
    seed = semilla # aqui se usa SU semilla
  )

  # genero el modelo
  # predecir clase_ternaria a partir del resto
  modelo <- rpart("clase_ternaria ~ .",
    data = dataset[fold == 1], # fold==1  es training,  el 70% de los datos
    xval = 0,
    control = param_basicos
  ) # aqui van los parametros del arbol

  # aplico el modelo a los datos de testing
  prediccion <- predict(modelo, # el modelo que genere recien
    dataset[fold == 2], # fold==2  es testing, el 30% de los datos
    type = "prob"
  ) # type= "prob"  es que devuelva la probabilidad

  # prediccion es una matriz con TRES columnas,
  #  llamadas "BAJA+1", "BAJA+2"  y "CONTINUA"
  # cada columna es el vector de probabilidades


  # calculo la ganancia en testing  qu es fold==2
  ganancia_test <- dataset[
    fold == 2,
    sum(ifelse(prediccion[, "BAJA+2"] > 0.025,
      ifelse(clase_ternaria == "BAJA+2", 117000, -3000),
      0
    ))
  ]

  # escalo la ganancia como si fuera todo el dataset
  ganancia_test_normalizada <- ganancia_test / (( 100 - PARAM$training_pct ) / 100 )

  return(
    c( list("semilla" = semilla),
      param_basicos,
      list( "ganancia_test" = ganancia_test_normalizada )
     )
  )
}

In [6]:
ArbolesMontecarlo <- function(semillas, param_basicos) {

  # la funcion mcmapply  llama a la funcion ArbolEstimarGanancia
  #  tantas veces como valores tenga el vector  PARAM$semillas
  salida <- mcmapply(ArbolEstimarGanancia,
    semillas, # paso el vector de semillas
    MoreArgs = list(PARAM$training_pct, param_basicos), # aqui paso el segundo parametro
    SIMPLIFY = FALSE,
    mc.cores = detectCores()
  )

  return(salida)
}

In [7]:
# carpeta de trabajo
# por fabor cambiar numero de experimento si se cambia el loop principal
setwd("/content/buckets/b1/exp")
experimento <- "HT2900"
dir.create(experimento, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento ))

In [8]:
# lectura del dataset
dataset <- fread("/content/datasets/dataset_pequeno.csv")

# trabajo solo con los datos con clase, es decir 202107
dataset <- dataset[clase_ternaria != ""]

In [9]:
# genero numeros primos
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia) # inicializo
# me quedo con PARAM$qsemillas   semillas
PARAM$semillas <- sample(primos, PARAM$qsemillas )

In [10]:
# genero la data.table donde van los resultados detallados del Grid Search
# un registro para cada combinacion de < semilla, parametros >

if(file.exists("autogridsearch_detalle.txt")){
  tb_grid_search_detalle <- fread("autogridsearch_detalle.txt")
}else{
  tb_grid_search_detalle <- data.table(
    semilla = integer(),
    cp = numeric(),
    maxdepth = integer(),
   minsplit = integer(),
    minbucket = integer(),
    ganancia_test = numeric()
  )
}

nrow( tb_grid_search_detalle )

[1] 0

In [11]:
# data.table threads (0 = auto use all)
data.table::setDTthreads(0)
data.table::getDTthreads()

[1] 8

Esta es la parte del código que usted debe expandir a TODOS los hiperparámetros de rpart,
<br>ya que actualmente apenas recorre  maxdepth y  minsplit  dejando fijos  cp=-0.5  y minbucket=5

Now that you have the top 100 parameter combinations, you can analyze the ranges of `cp`, `maxdepth`, `minsplit`, and `minbucket` that appear most frequently among these high-performing combinations. This analysis will help you determine the best ranges for manual reproduction or further investigation.

# 4.  Análisis de resultados de Grid Search

La salida de la corrida anterior queda en ~/buckets/b1/exp/HT2900  que corresponde a su Google Drive
<br>HT significa Hyperparameter Tuning
<br>El Grid Search es un método de fuerza bruta de un altísimo costo computacional.
<br>Queremos ver si es posible crear un algoritmo de optimización de hiperparámetros que se ahorre recorrer ciertas porciones muy malas del espacio de búsqueda. Algo del estilo “cada vez que pruebo una combinación de hiperparámetros donde  cp > 1 , la ganancia es muy mala, con lo cual ni vale la pena perder el tiempo explorando en esa region”


<br>Levante el archivo de salida gridsearch.txt  a una planilla tipo Excel y analícelo detenidamente
<br>Ordene por ganancia_mean descendente
<br>
<br>El de mayor ganancia_mean  decimos que es el primero del ranking
En Zulip, correspondiente channel  #Tarea Hogar 02 , topic Analisis Grid Search   intente contestar estas preguntas:

* ¿Qué combinaciones de hiperparámetros poseen una ganancia muy buena?
* ¿Hay algun hiperparámetro que para cierto valor siempre genera una ganancia muy mala, a independientemente de lo que valgan los otros hiperparámetros ?
* ¿Que combinaciones de hiperparámetros es pésima y hubiera sido bueno ahorrarse esas corridas ?

( tiempo estimado 30 minutos, dificultad media )

## Track the best hyperparameters and gain

### Subtask:
Modify the code to keep track of the best hyperparameters found during the optimization process and their corresponding gain.

In [12]:
# Install and load the rBayesianOptimization package if not already available
if (!require("rBayesianOptimization")) install.packages("rBayesianOptimization")
require("rBayesianOptimization")

Loading required package: rBayesianOptimization



In [13]:
# Define the objective function to maximize
EstimarGananciaHiperparametros <- function(cp, maxdepth, minsplit, minbucket) {
  # Ensure integer hyperparameters are treated as integers
  maxdepth <- as.integer(maxdepth)
  minsplit <- as.integer(minsplit)
  minbucket <- as.integer(minbucket)

  # Round cp to the nearest 0.1
  cp_rounded <- round(cp * 10) / 10

  # Set the hyperparameters for rpart control
  param_basicos <- list(
    cp = cp_rounded,
    maxdepth = maxdepth,
    minsplit = minsplit,
    minbucket = minbucket
  )

  # Run Monte Carlo simulation with the given hyperparameters
  ganancias <- ArbolesMontecarlo(PARAM$semillas, param_basicos)

  # Calculate the mean gain
  ganancia_promedio <- mean(sapply(ganancias, "[[", "ganancia_test"))

  # Return the mean gain (the objective function value)
  return(list(Score = ganancia_promedio, Pred = 0)) # BayesianOptimization expects a list with Score and Pred
}

In [ ]:
# Define the bounds for each hyperparameter, including negative values for cp
bounds <- list(
  cp = c(-1, 0.1), # cp can be between -0.01 and 0.01
  maxdepth = c(3L, 15L), # maxdepth can be between 3 and 15 (integer)
  minsplit = c(2L, 3000L), # minsplit can be between 2 and 2000 (integer)
  minbucket = c(1L, 1000L) # minbucket can be between 1 and 1000 (integer)
)

# Perform Bayesian Optimization
set.seed(PARAM$semilla_primigenia) # Set seed for reproducibility
bayes_opt_result <- BayesianOptimization(
  FUN = EstimarGananciaHiperparametros, # The objective function to maximize
  bounds = bounds, # The hyperparameter bounds
  init_points = 20L, # Number of initial random points
  n_iter = 80, # Number of iterations for optimization, increased from 30 to 90
  acq = "ei", # Acquisition function (Upper Confidence Bound)
  #kappa = 2.576, # Kappa value for UCB (exploratory vs exploitative trade-off)
  eps = 0.01, # Epsilon for epsilon-greedy acquisition function
  verbose = TRUE # Print optimization progress
)

# Print the best parameters found
print(bayes_opt_result$Best_Par)
print(bayes_opt_result$Best_Value)

In [ ]:
# Get the results from the Bayesian optimization
optimization_results <- bayes_opt_result$History

# Display column names to identify the gain column
print(colnames(optimization_results))

# Convert the results to a data.table
tb_optimization_results <- as.data.table(optimization_results)

# Order the results by the gain (Value) in descending order
setorder(tb_optimization_results, -Value)

# Display the top 100 parameter combinations and their gain
print(tb_optimization_results[1:100])


In [ ]:
# genero un id a la tabla
tb_optimization_results[, id := .I ]

fwrite( tb_optimization_results,
  file = "BayesGridSearch.txt",
  sep = "\t"
)

# Task
Improve the Bayesian Optimization process to find better hyperparameters for the decision tree model, aiming for higher average gains, by increasing the number of iterations, refining hyperparameter bounds, experimenting with different acquisition functions, increasing the number of initial points, considering cross-validation, increasing the number of seeds for Monte Carlo simulation, and ensuring effective parallel processing.

## Analyze current results

### Subtask:
Review the current Bayesian Optimization results (`bayes_opt_result`) to understand the range of hyperparameters explored and the resulting gains. This can help identify if the optimization is converging or if it's still exploring promising regions.


**Reasoning**:
Display the best parameters and value from the Bayesian optimization results and show the history of the optimization to understand the explored hyperparameter space and resulting gains.



In [ ]:
print("Best Parameters Found:")
print(bayes_opt_result$Best_Par)
print("\nBest Value (Mean Gain):")
print(bayes_opt_result$Best_Value)

print("\nOptimization History (showing the first 10 rows):")
print(tb_optimization_results[1:10])

**Reasoning**:
Analyze the range of explored hyperparameters and their relationship with the gain from the optimization history to understand the effectiveness of the current optimization process.



In [ ]:
# Analyze the distribution of hyperparameters in the top performing combinations
print("\nSummary statistics for hyperparameters in the top 100 combinations:")
print(summary(tb_optimization_results[1:100]))

# Look for trends in hyperparameter values as gain increases
# This is a basic visual inspection, more detailed analysis might involve plotting
print("\nHyperparameter values and gain for the top 20 combinations:")
print(tb_optimization_results[1:100, c("cp", "maxdepth", "minsplit", "minbucket", "Value")])

## Increase the number of iterations

### Subtask:
Increase the number of iterations (`n_iter`) in the `BayesianOptimization` function to allow for a more extensive exploration of the hyperparameter space.


**Reasoning**:
Increase the number of iterations in the BayesianOptimization function to explore the hyperparameter space more extensively.



In [ ]:
# Define the objective function to maximize
EstimarGananciaHiperparametros <- function(cp, maxdepth, minsplit, minbucket) {
  # Ensure integer hyperparameters are treated as integers
  maxdepth <- as.integer(maxdepth)
  minsplit <- as.integer(minsplit)
  minbucket <- as.integer(minbucket)

  # Set the hyperparameters for rpart control
  param_basicos <- list(
    cp = cp,
    maxdepth = maxdepth,
    minsplit = minsplit,
    minbucket = minbucket
  )

  # Run Monte Carlo simulation with the given hyperparameters
  ganancias <- ArbolesMontecarlo(PARAM$semillas, param_basicos)

  # Calculate the mean gain
  ganancia_promedio <- mean(sapply(ganancias, "[[", "ganancia_test"))

  # Return the mean gain (the objective function value)
  return(list(Score = ganancia_promedio, Pred = 0)) # BayesianOptimization expects a list with Score and Pred
}


# Define the bounds for each hyperparameter, including negative values for cp
bounds <- list(
  cp = c(-1, 0.1), # cp can be between -0.01 and 0.01
  maxdepth = c(3L, 15L), # maxdepth can be between 3 and 15 (integer)
  minsplit = c(2L, 3000L), # minsplit can be between 2 and 2000 (integer)
  minbucket = c(1L, 1000L) # minbucket can be between 1 and 1000 (integer)
)

# Perform Bayesian Optimization
set.seed(PARAM$semilla_primigenia) # Set seed for reproducibility
bayes_opt_result <- BayesianOptimization(
  FUN = EstimarGananciaHiperparametros, # The objective function to maximize
  bounds = bounds, # The hyperparameter bounds
  init_points = 20L, # Number of initial random points
  n_iter = 80, # Number of iterations for optimization, increased from 30 to 90
  acq = "ei", # Acquisition function (Upper Confidence Bound)
  #kappa = 2.576, # Kappa value for UCB (exploratory vs exploitative trade-off)
  eps = 0.01, # Epsilon for epsilon-greedy acquisition function
  verbose = TRUE # Print optimization progress
)

# Print the best parameters found
print(bayes_opt_result$Best_Par)
print(bayes_opt_result$Best_Value)

In [ ]:
# 1) Pull BO history
hist <- as.data.table(bayes_opt_result$History)

# 2) Normalize columns present in history
has_ratio <- "minbucket_ratio" %in% names(hist)

# Cast integer-like params
int_cols <- intersect(c("maxdepth","minsplit","minbucket"), names(hist))
if (length(int_cols)) hist[, (int_cols) := lapply(.SD, as.integer), .SDcols = int_cols]

# Optional: derive minbucket from ratio if BO used minbucket_ratio
if (has_ratio && !("minbucket" %in% names(hist))) {
  # Clamp to [1, floor(minsplit/2)]
  hist[, minbucket := pmax(1L, as.integer(round(minsplit * minbucket_ratio)))]
  hist[, minbucket := pmin(minbucket, as.integer(floor(minsplit / 2L)))]
}

# 3) For grouping, round cp a bit to merge near-duplicates produced by GP jitter
hist[, cp_round := round(cp, 6)]

# 4) Aggregate by unique combo -> mean and best observed Value
group_cols <- intersect(c("cp_round","maxdepth","minsplit","minbucket"), names(hist))
stopifnot(all(c("Value", group_cols) %in% names(hist)))

agg <- hist[, .(
  mean_ganancia = mean(Value, na.rm = TRUE),
  best_ganancia = max(Value, na.rm = TRUE),
  n_evals       = .N
), by = group_cols]

# 5) Recover original cp (unrounded) by taking the average of cp per group (optional)
if ("cp" %in% names(hist)) {
  cp_map <- hist[, .(cp = mean(cp, na.rm = TRUE)), by = .(cp_round)]
  agg <- cp_map[agg, on = "cp_round"]
  setcolorder(agg, c("cp","maxdepth","minsplit","minbucket","mean_ganancia","best_ganancia","n_evals","cp_round"))
  agg[, cp_round := NULL]
} else {
  setnames(agg, "cp_round", "cp")
}

# 6) Order and take top 100
setorder(agg, -mean_ganancia, -best_ganancia, n_evals)
top100 <- head(agg, 100)

# 7) Show and (optionally) write to disk
print(top100, nrows = 100)
data.table::fwrite(top100, "bo_top100_params.tsv", sep = "\t")
